In [ ]:
# 建立樞紐分析表
https://www.yinglinglow.com/blog/2018/04/29/xlwings-pivot-table

In [ ]:
VBA寫法： https://docs.microsoft.com/zh-tw/office/vba/api/excel.pivotcaches.create
ActiveWorkbook.PivotCaches.Create(SourceType:=xlExternal, SourceData:=ActiveWorkbook.Connections("Target Connection Name"), _ 
Version:=xlPivotTableVersion15).CreatePivotChart(ChartDestination:="Sheet1").Select

In [6]:
import win32com.client
Excel   = win32com.client.gencache.EnsureDispatch('Excel.Application') 
# Excel = win32com.client.Dispatch('Excel.Application')

win32c = win32com.client.constants

wb = Excel.Workbooks.Add()
Sheet1 = wb.Worksheets("Sheet1")

TestData = [['Country','Name','Gender','Sign','Amount'],
             ['CH','Max' ,'M','Plus',123.4567],
             ['CH','Max' ,'M','Minus',-23.4567],
             ['CH','Max' ,'M','Plus',12.2314],
             ['CH','Max' ,'M','Minus',-2.2314],
             ['CH','Sam' ,'M','Plus',453.7685],
             ['CH','Sam' ,'M','Minus',-53.7685],
             ['CH','Sara','F','Plus',777.666],
             ['CH','Sara','F','Minus',-77.666],
             ['DE','Hans','M','Plus',345.088],
             ['DE','Hans','M','Minus',-45.088],
             ['DE','Paul','M','Plus',222.455],
             ['DE','Paul','M','Minus',-22.455]]

for i, TestDataRow in enumerate(TestData):
    for j, TestDataItem in enumerate(TestDataRow):
        Sheet1.Cells(i+2,j+4).Value = TestDataItem

cl1 = Sheet1.Cells(2,4)  #最左上格
cl2 = Sheet1.Cells(2+len(TestData)-1,4+len(TestData[0])-1)  #最右下格
PivotSourceRange = Sheet1.Range(cl1,cl2)  #左上~右下:要樞紐的全範圍

PivotSourceRange.Select()

Sheet2 = wb.Worksheets(2)
cl3=Sheet2.Cells(4,1)
PivotTargetRange=  Sheet2.Range(cl3,cl3)
PivotTableName = 'ReportPivotTable'

#建立樞紐分析表時，一定會有一個樞紐分析快取(Pivot Cache)的記憶體配置
#https://docs.microsoft.com/zh-tw/office/vba/api/excel.pivotcaches.create
PivotCache = wb.PivotCaches().Create(SourceType=win32c.xlDatabase, SourceData=PivotSourceRange)

#在快取之下才能建立樞紐分析表
#https://docs.microsoft.com/zh-tw/office/vba/api/excel.pivotcache.createpivottable
PivotTable = PivotCache.CreatePivotTable(TableDestination=PivotTargetRange, TableName=PivotTableName)

#欄位設定(PivotField物件):Excel中選擇要新增到樞紐報表的位置動作
    #Orientation(定位):顯示欄位要放在樞紐的位置  https://docs.microsoft.com/zh-tw/office/vba/api/excel.pivotfield.orientation
    #Position(位置):同一Orientation(定位)下欄位的先後順序  https://docs.microsoft.com/zh-tw/office/vba/api/excel.pivotfield.position
    #CurrentPage(當前頁面):報表篩選(左上)中進行篩選  https://docs.microsoft.com/zh-tw/office/vba/api/excel.pivotfield.currentpage
    #Subtotals(小計):傳回或設定與指定欄位一同顯示的小計。 僅對非資料欄位有效  https://docs.microsoft.com/zh-tw/office/vba/api/excel.pivotfield.subtotals
PivotTable.PivotFields('Name').Orientation = win32c.xlRowField  #'Name'定位在列標籤(左下)
PivotTable.PivotFields('Name').Position = 1   #在列標籤(左下)的第一位置(1為最先，無0)

PivotTable.PivotFields('Gender').Orientation = win32c.xlPageField  #'Gender'定位在報表篩選(左上)
PivotTable.PivotFields('Gender').Position = 1 #在列標籤(左上)的第一位置(1為最先，無0)
PivotTable.PivotFields('Gender').CurrentPage = 'M'  #報表篩選(左上)中進行篩選

PivotTable.PivotFields('Country').Orientation = win32c.xlColumnField  #'Country'定位在欄標籤(右上)
PivotTable.PivotFields('Country').Position = 1  #在欄標籤(右上)的第一位置(1為最先，無0)
PivotTable.PivotFields('Country').Subtotals = [False, False, False, False, False, False, False, False, False, False, False, False]
PivotTable.PivotFields('Sign').Orientation = win32c.xlColumnField    #'Sign'定位在欄標籤(右上)
PivotTable.PivotFields('Sign').Position = 2     #在欄標籤(右上)的第二位置(1為最先，無0)


DataField = PivotTable.AddDataField(PivotTable.PivotFields('Amount'))  #加總區(右下)的值設定為'Amount'
DataField.NumberFormat = '0.00'  #加總區(右下)的值欄位設定格式
 
Excel.Visible = 1

#wb.SaveAs('ranges_and_offsets.xlsx')
#Excel.Application.Quit()

In [5]:
for i, TestDataRow in enumerate(TestData):
    print(i, TestDataRow)

0 ['Country', 'Name', 'Gender', 'Sign', 'Amount']
1 ['CH', 'Max', 'M', 'Plus', 123.4567]
2 ['CH', 'Max', 'M', 'Minus', -23.4567]
3 ['CH', 'Max', 'M', 'Plus', 12.2314]
4 ['CH', 'Max', 'M', 'Minus', -2.2314]
5 ['CH', 'Sam', 'M', 'Plus', 453.7685]
6 ['CH', 'Sam', 'M', 'Minus', -53.7685]
7 ['CH', 'Sara', 'F', 'Plus', 777.666]
8 ['CH', 'Sara', 'F', 'Minus', -77.666]
9 ['DE', 'Hans', 'M', 'Plus', 345.088]
10 ['DE', 'Hans', 'M', 'Minus', -45.088]
11 ['DE', 'Paul', 'M', 'Plus', 222.455]
12 ['DE', 'Paul', 'M', 'Minus', -22.455]


In [6]:
for j, TestDataItem in enumerate(TestDataRow):
    print(j, TestDataItem)

0 DE
1 Paul
2 M
3 Minus
4 -22.455


In [7]:
Sheet2 = wb.Worksheets("Sheet2")

In [15]:
#包含指定之範圍儲存格的樞紐分析報表
Sheet2.Range("A2").PivotTable.Name

'ReportPivotTable'

In [22]:
#樞紐分析表報表中的儲存格
Sheet2.Range("D5").PivotCell

<win32com.gen_py.Microsoft Excel 12.0 Object Library.PivotCell instance at 0x92700568>

In [16]:
#代表包含指定之範圍左上角儲存格的樞紐分析表欄位。
Sheet2.Range("D5").PivotField.Name

'Country'

In [18]:
import win32com.client
Excel   = win32com.client.gencache.EnsureDispatch('Excel.Application') 
# Excel = win32com.client.Dispatch('Excel.Application')

win32c = win32com.client.constants

Excel.Visible = 1

wb = Excel.Workbooks.Open(r'C:\Users\11004076\Documents\Python Scripts\4_Automation\20200324-進銷存-SF4_Vietnam_Silicom.xlsx')  #開啟現有活頁簿
Sheet1 = wb.Worksheets("raw")

PivotSourceRange = Sheet1.Range("A1:I1234")

Sheet2 = wb.Worksheets("pvt")
PivotTargetRange=  Sheet2.Range("C5")
PivotTableName = 'ReportPivotTable'

In [19]:
#建立樞紐分析表時，一定會有一個樞紐分析快取(Pivot Cache)的記憶體配置
PivotCache = wb.PivotCaches().Create(SourceType=win32c.xlDatabase, SourceData=PivotSourceRange)

#在快取之下才能建立樞紐分析表
PivotTable = PivotCache.CreatePivotTable(TableDestination=PivotTargetRange, TableName=PivotTableName)

#欄位設定(PivotField物件):Excel中選擇要新增到樞紐報表的位置動作
PivotTable.PivotFields('MediumCat').Orientation = win32c.xlRowField
PivotTable.PivotFields('MediumCat').Position = 1
PivotTable.PivotFields('Sls CD4 Desc#').Orientation = win32c.xlRowField
PivotTable.PivotFields('Sls CD4 Desc#').Position = 2
PivotTable.PivotFields('2nd Item Number').Orientation = win32c.xlRowField
PivotTable.PivotFields('2nd Item Number').Position = 3
PivotTable.PivotFields('Product Desc#').Orientation = win32c.xlRowField
PivotTable.PivotFields('Product Desc#').Position = 4

PivotTable.PivotFields('ISS').Orientation = win32c.xlColumnField
PivotTable.PivotFields('ISS').Position = 1
PivotTable.PivotFields('ISS').Subtotals = [False, False, False, False, False, False, False, False, False, False, False, False]
PivotTable.PivotFields('Year').Orientation = win32c.xlColumnField
PivotTable.PivotFields('Year').Position = 2
PivotTable.PivotFields('Month').Orientation = win32c.xlColumnField
PivotTable.PivotFields('Month').Position = 3
PivotTable.PivotFields('Month').Subtotals = [False, False, False, False, False, False, False, False, False, False, False, False]

DataField = PivotTable.AddDataField(PivotTable.PivotFields('Qty'))
#DataField = PivotTable.PivotFields('Qty').Orientation = win32c.xlDataField
DataField.NumberFormat = '0.00'

PivotTable.PivotFields('LargeCat').Orientation = win32c.xlPageField
PivotTable.PivotFields('LargeCat').Position = 1
PivotTable.PivotFields('LargeCat').CurrentPage = 'Mouse'

#wb.SaveAs('ranges_and_offsets.xlsx')
#Excel.Application.Quit()
